In [41]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

In [42]:
df = pd.read_csv("/kaggle/input/refined-dataset2/best_features.csv")
label = pd.read_csv("/kaggle/input/refined-dataset2/data.csv")
df

,winsorized_Adult_Mortality,winsorized_Schooling,winsorized_Diphtheria,winsorized_Percentage_Exp,winsorized_Infant_Deaths,winsorized_GDP,winsorized_Income_Comp_Of_Resources,winsorized_Tot_Exp
0,263.0,10.1,65.0,71.279624,57,584.259210,0.479,8.16
1,271.0,10.0,62.0,73.523582,57,612.696514,0.476,8.18
2,268.0,9.9,64.0,73.219243,57,631.744976,0.470,8.13
3,272.0,9.8,67.0,78.184215,57,669.959000,0.463,8.52
4,275.0,9.5,68.0,7.097109,57,63.537231,0.454,7.87
...,...,...,...,...,...,...,...,...
2909,453.0,9.2,65.0,0.000000,27,454.366654,0.407,7.13
2910,453.0,9.5,68.0,0.000000,26,453.351155,0.418,6.52
2911,73.0,10.0,71.0,0.000000,25,57.348340,0.427,6.53
2912,453.0,9.8,75.0,0.000000,25,548.587312,0.427,6.16


In [ ]:
sc = ['winsorized_Adult_Mortality',
 'winsorized_Schooling',
 'winsorized_Diphtheria',
 'winsorized_Percentage_Exp',
 'winsorized_Infant_Deaths',
 'winsorized_GDP',
 'winsorized_Income_Comp_Of_Resources',
 'winsorized_Tot_Exp',
 'winsorized_thinness_5to9_years',
 'winsorized_thinness_1to19_years']

In [53]:
# example of a normalization
from numpy import asarray
from sklearn.preprocessing import MinMaxScaler
from numpy import asarray
from sklearn.preprocessing import StandardScaler
# define data
for i in sc:
# define standard scaler

    scaler = MinMaxScaler()

    df[[i]] = scaler.fit_transform(df[[i]])

1.0

In [43]:
numerical_columns = [x for x in list(df.columns) if x.startswith("winsorized")]
numerical_columns

['winsorized_Adult_Mortality',
 'winsorized_Schooling',
 'winsorized_Diphtheria',
 'winsorized_Percentage_Exp',
 'winsorized_Infant_Deaths',
 'winsorized_GDP',
 'winsorized_Income_Comp_Of_Resources',
 'winsorized_Tot_Exp']

In [44]:
numerical_columns = list(set(numerical_columns) - set("winsorized_Life_Expectancy"))
y = "winsorized_Life_Expectancy"
X_train = np.array(df[numerical_columns])
y_train = np.array(label[y])
prediction = np.array(df[numerical_columns].iloc[0].values)

In [45]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_train,y_train ,
                                   random_state=1100, 
                                   test_size=0.25, 
                                   shuffle=True)

In [46]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
tscv = TimeSeriesSplit(n_splits=5)
i = 1
score = []
for tr_index, val_index in tscv.split(X_train):
    X_tr, X_val = X_train[tr_index], X_train[val_index]
    y_tr, y_val = y_train[tr_index], y_train[val_index]
    for mf in np.linspace(100, 150, 6):
        for ne in np.linspace(50, 100, 6):
            for md in np.linspace(20, 40, 5):
                for msl in np.linspace(30, 100, 8):
                    rfr = RandomForestRegressor(
                        max_features=int(mf/100),
                        n_estimators=int(ne),
                        max_depth=int(md),
                        min_samples_leaf=int(msl))
                    rfr.fit(X_tr, y_tr)
                    score.append([i,
                                  mf, 
                                  ne,
                                  md, 
                                  msl, 
                                  rfr.score(X_val, y_val), r2_score(rfr.predict(X_val ) , y_val)])
    i += 1

In [47]:
score[0]
max(score , key = lambda x : x[5])

[5, 150.0, 50.0, 30.0, 30.0, 0.8201195825534491, 0.7144894140545093]

we get the highest accuracy for max_features = 1.1 , n_estimators = 60.0 , max_depth = 25.0 , min_samples_leaf = 30.0

In [48]:
model = RandomForestRegressor(  max_features=int(110.0/100),
                        n_estimators=int(60.0),
                        max_depth=int(25.0),
                        min_samples_leaf=int(30.0))

In [49]:
from sklearn.metrics import r2_score
max_score = 0
for tr_index, val_index in tscv.split(X_train):
    X_tr, X_val = X_train[tr_index], X_train[val_index]
    y_tr, y_val = y_train[tr_index], y_train[val_index]
    rfr = RandomForestRegressor(  max_features=int(110.0/100),
        n_estimators=int(60.0),
        max_depth=int(25.0),
        min_samples_leaf=int(30.0))
    rfr.fit(X_tr, y_tr)
    score.append([i,
                  mf, 
                  ne,
                  md, 
                  msl, 
                  rfr.score(X_val, y_val)])
    if rfr.score(X_val, y_val) > max_score:
        model = rfr
                
                

In [50]:
rfr.predict([prediction])

array([60.18870432])

In [51]:
import pickle
pickle.dump(rfr, open('model_baseline_RandomForestRegressor.pkl','wb'))